# Generate_muropeptide_library.ipynb
Created by: Irnov Irnov (Jacobs-Wagner lab, Stanford University), 2020-2023

### Aim: 
Generate a library of predicted masses for Borrelia burgdorferi peptidoglycan fragments (muropeptides)

### Input:
Table of amino acids and glycan species with their corresponding masses. See 'Muropeptide_mass_table.csv' 

### Output:
List of possible peptidoglycan monomers and dimers with associated masses. Muropeptide prediction is based on B. burgdorferi PG composition described in Jutras et al, 2019, PNAS. See 'Muropeptide_list_and_mass.csv'

### Output species annotation (column 1 in the output file):
A = alanine <br>
E = glutamate <br>
Q = glutamine <br>
K = lysine <br>
O = ornithine <br>
B = meso-DAP <br>
J = MurNac <br>
U = Anhydro-MurNac <br>
X = GlcNac-MurNac <br>
Z = GlcNac-AnhydroMurNac <br>
H = terminal HexNac (Borrelia PG, see Jutras et al, 2019)

### Requirements:
Python (3.8.8), Matplotlib (3.4.1), Seaborn (0.11.1), Numpy (1.20.1), Pandas (1.2.4)

In [2]:
# ------------ DEFINE COMPOSITION FOR GLYCAN, PEPTIDE, CROSSBRIDGE SEQUENCE ------------

'''
Vector to hold the peptide sequence (data type is a list of list).
Each position in the vector will hold all possible sequence variants in a vector.
Each position correspond to position 1 to 5 in the peptide sidechain for PG.

A = alanine
E = glutamate
Q = glutamine
K = lysine
O = ornithine
B = meso-DAP
'''
peptide = [
    ['A'],
    ['E'],
    ['O'],
    ['A'],
    ['A']
]


'''
Vector to hold the glycan sequence. 
Only limited to 4 possibility for now: 
    J = MurNac
    U = Anhydro-MurNac
    X = GlcNac-MurNac
    Z = GlcNac-AnhydroMurNac
'''
glycan = ['J', 'U', 'X', 'Z']


'''
Vector to hold the cross-bridge sequence
G = glycine

'''
cross = ['G']

'''
Other masses
'''
Hy = 1.007276     #hydrogen
Na = 22.98976928  #sodium adduct
water = 18.0105647

In [3]:
# ------------------------ FUNCTIONS ------------------------

import sys
import os
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


# This function will concatenate a string 'ext' to all sequences in a list 'seq'
def extend_seq(seq, ext):
    
    #if the input is a string, then simply extend the string
    if isinstance(seq, str) and isinstance(ext, str): 
        return seq+ext
    
    #if the input is a list, then add extra string to each of the sequence in the list
    elif isinstance(seq, list) and isinstance(ext, str):
        
        new_seq=[]
        
        if len(seq) == 0: new_seq.append(ext)
        
        for i in range(0,len(seq)):
            new_seq.append(seq[i] + ext)
        
        return new_seq
    
    #otherwise, print error and exit
    else:
        print('Input Error')
        exit(1)

'''------------------------------------------------'''

# Generate all possible combinations for PG peptide fragments
def generate_peptide_variants(peptide):
    
    #variable to hold the new sequences
    pept_list = []
    
    #list to hold the previous sequences, which will be extended with variation at each position
    prev = []

    #loop through each position in the peptide
    #extend the peptide sequence one at a time
    for x in range (0, len(peptide)):

        #temporary vector to hold the new peptide sequences
        temp = []

        #loop through options/variants at each peptide position    
        for y in range(0, len(peptide[x])):

            #call extend_seq function to extend each peptide by one sequence
            temp = temp + extend_seq(prev,peptide[x][y])

        #assign the current extended peptide sequences to 'prev' parameter, 
        #which will be extended in the next round
        prev = temp

        #add the extended peptide sequences to the collection of all peptides
        pept_list = pept_list + temp
        
    return pept_list

'''------------------------------------------------'''
#Add glycan sequence
#Here, I treat monosaccharide or disaccharide each independently. 
#See single-letter code above for each glycan

def add_glycan(seq, glycan):
    new_seq = seq

    #loop through all peptide sequences and add glycan
    for p in pept_seq:
        new_seq = new_seq + extend_seq(glycan, p)
  
    #add glycan only sequences to the list
    new_seq = new_seq + glycan
    
    return new_seq

'''------------------------------------------------'''
#Add crossbridge sequence
#This function will require the organism to be specified. Default = Borrelia
#Crossbridge sequence is specified as a string in variable 'cross'
#Future version will include crossbridge sequence for other organisms

def add_crossbridge(seq, cross, organism='Borrelia'):
    
    new_seq = seq
    
    if organism=="Borrelia":
        
        #For now, only add the crossbridge amino acid for Borrelia PG (with ornithine)
        #Also add HexNac here, before the GlcNac
        #This will not take into account whether the crossbridge position (3rd or 4th) - simply add a Glycine
        temp = []

        #get all sequences with ornithine 'O' and at least 3 amino acids -- this is Borrelia specific
        for pg in seq:
            if 'AEO' in pg: temp.append(pg)

        #add the extra glycine crossbridge
        temp = extend_seq(temp, cross)
        
        new_seq = new_seq + temp 
        
        #add HexNac (H) to anything that starts with Z or X (ie. with GlcNac-MurNac or GlcNac-AnhMurNac)
        temp = []
        for pg in new_seq:
            if pg.startswith('X') | pg.startswith('Z'):
                pg = 'H' + pg
                temp.append(pg)
        
        new_seq = new_seq + temp
 
    return new_seq

'''------------------------------------------------'''
#create dimer sequences
#dimer through the crossbridge, only allow 3-4 linkage based on Jutras et_al for Borrelia
#organism name can be specified for specific dimer requirements (Default = 'Borrelia')
#Here, I inlcuded E. coli dimers, but future version will add dimers composition for other organisms

def add_dimers(seq, organism='Borrelia'):
    
    new_seq = seq
    
    #create variables to hold temporary values
    temp_left = []
    temp_right= []
    temp = []
    dimers = []
        
    if organism=="Borrelia":
      
        for pg in seq:
            #make sure that the left side fragments will at least have 3 amino acids (ie., tripeptide)
            #and a glycine crossbridge
            #this will include tetra and pentapeptide as well
            if ('AEO' in pg) & ('G' in pg):
                temp_left.append(pg)

            #only include fragment as the right side fragment if it contains at least 3 amino acids (tripeptide)
            #CHANGED ON 11/08/2021 (IRNOV) - From requiring tetrapeptide (AEOA) to tripeptide (AEO)
            #This modification will allow for detection of PG dimer through sugar backbone like molecule #5
            #in Jutras et al PNAS 2019 
            if 'AEO' in pg:
                temp_right.append(pg)

        #go through each possible right side fragments and add to the left fragment
        for right_pg in temp_right:
            right_pg = '-' + right_pg
            dimers = dimers + extend_seq(temp_left, right_pg)

    if organism=="Ecoli":
      
        for pg in seq:
            #make sure that the left and right side fragments will at least have 3 amino acids (ie., tripeptide)
            #this will include tetra and pentapeptide as well
            if 'AEB' in pg:
                temp_left.append(pg)
                temp_right.append(pg)
                    

        #go through each possible right side fragments and add to the left fragment
        for right_pg in temp_right:
            right_pg = '-' + right_pg
            dimers = dimers + extend_seq(temp_left, right_pg)
    
    #trim the dimer list by removing dimers with flipped sequence
    #for example: ZAEOAAG-XAEOAAG and XAEOAAG-ZAEOAAG
    #both can't be differentiated based on mass alone
    for seq in dimers:
        two_halves = seq.split('-')
        
        #IF LEFT SIDE NOT EQUAL TO RIGHT SIDE, search for the flipped sequence and
        #remove if exist
        if two_halves[0] != two_halves[1]:
            flipped_seq = two_halves[1]+'-'+two_halves[0]
        
            if flipped_seq in dimers: dimers.remove(seq)
    
    #append dimer sequences and return new list
    new_seq = new_seq + dimers
    return new_seq
    
'''------------------------------------------------'''
#Generate list of masses based for each muropeptide
#Need input file in csv format containing the single-letter code and corresponding mass
#For this paper, see: 'Muropeptide_mass_table.csv'
#First column = single-letter code for the molecule
#Second column = name
#Third column = mass

def create_code_mass_conversion(input_file, delim = ','):
    #Use pandas to read muropeptide mass information from file
    mass_table = pd.read_csv(input_file, delimiter=delim)

    #create a dictionary to convert identity code into mass
    #first, turn data from dataframe into list format
    code = list(mass_table.iloc[:,0])
    mass = list(mass_table.iloc[:,2])

    #convert to dictionary
    code_to_mass = convert_to_dict (code, mass)

    return code_to_mass

#second, create a function that takes two list and generate a dictionary
def convert_to_dict(X, Y):
    dct = {X[i]: Y[i] for i in range (0, len(X))}
    return dct

'''------------------------------------------------'''
#Turn each muropeptide sequence to mass
#The input is a single sequence ('sequence') and a dictionary to convert character into mass

def get_mass(sequence, dct):
    mass = 0
    
    #remove the dash '-' sign if present. This is not necessary for calculating mass
    sequence = sequence.replace('-','')
        
    #add up the mass for each compound (ie., each letter)
    for c in sequence:
        mass = mass + dct[c]
    
    #substract the mass of water for each addition (one less than the lenght of the sequence)
    mass = mass - ((len(sequence)-1)*water)
    
    return mass


In [5]:
'''
MAIN FUNCTION

Here, we will generate all possible variations of Borrelia muropeptides using functions described above. 
Save list of muropeptides to an output file. 
'''

#USER SPECIFIED VARIABLES
#Replace with the appropriate path and file names
current_dir = '/Users/irnov/Desktop/Experiments/Data/LCMS/'
input_mass_file = 'Muropeptide_mass_table.csv'
output_file = 'Muropeptide_list_and_mass_.csv'

#Move to specified working directory
os.chdir(current_dir)

pept_seq = []
PG_seq = []

#create all possible peptide sequences
pept_seq = generate_peptide_variants(peptide)

#add glycan sequences
PG_seq = add_glycan(pept_seq,glycan)

#add crossbridge
PG_seq = add_crossbridge (PG_seq, cross[0], organism='Borrelia')

#add dimers
PG_seq = add_dimers (PG_seq, organism='Borrelia')

#read file containing the conversion info from symbol to mass
code_to_mass = create_code_mass_conversion(input_mass_file)

#Add the code/amino acids/individual sugar to the sequences so I can search their masses as well
PG_seq = PG_seq + list(code_to_mass.keys())

#calculate the exact mass for individual PG sequence
masses = []
for pg in PG_seq:
    m = get_mass(pg, code_to_mass)
    masses.append(m)
    
#create a dataframe by zip-ping the list together
master_table = pd.DataFrame(list(zip(PG_seq, masses)), columns = ['Species','Mass'])

#add m/z for hydrogen and soidum adduct
#In this case, I will create masses for M+H, M+2H, M+3H, M-H, M-2H, M+Na
master_table['mz_plus_1'] = master_table.Mass + Hy
master_table['mz_plus_2'] = (master_table.Mass + 2*Hy)/2
master_table['mz_plus_3'] = (master_table.Mass + 3*Hy)/3
master_table['mz_minus_1'] = (master_table.Mass - Hy)
master_table['mz_minus_2'] = (master_table.Mass - 2*Hy)/2
master_table['mz_plus_Na'] = master_table.Mass + Na

# Save dataframe into a csv file
master_table.to_csv(output_file)